In [72]:
import ta
import time
import math
import pprint
import asyncio
import datetime
import itertools
import pandas as pd
import SecretConfig
import ccxt.pro as ccxt
from IPython.display import clear_output

In [73]:
# §§§§§§§§§§

await exchange.close()

# §§§§§§§§§§

In [74]:
exchange = ccxt.bitget({
    "apiKey":SecretConfig.APIKey,
    "secret":SecretConfig.SecretKey,
    'password': SecretConfig.Passphrase,
    "options": {'defaultType': 'swap', 'adjustForTimeDifference' : True},
    "timeout": 60000,
    "enableRateLimit": True,
})

In [75]:
# LIVE PRICE ET MARKETS

async def live_prices() :

    global last_20
    global tickers
    global swap_markets
  
    tickers = await exchange.fetch_tickers()

    await asyncio.sleep(0.005)

    timestamp = list(tickers.values())[0]['timestamp']
    index = [datetime.datetime.fromtimestamp(timestamp/1000).replace(microsecond=0)]

    data = {}

    for symbol, ticker in tickers.items():

        data[symbol +  " last"] = [ticker['last']]
        data[symbol +  " ask"] = [ticker['ask']]
        data[symbol +  " bid"] = [ticker['bid']]

    last_prices = pd.DataFrame(data, index=index)
    last_20 = pd.concat([last_20, last_prices])
    last_20 = last_20.tail( wind_EMA2 + wind_Var_EMA2 + wind_Div_EMA1_2 + wind_EMA2 +30)

    markets = await exchange.load_markets()

    await asyncio.sleep(0.005)

    swap_markets = {symbol: market for symbol, market in markets.items() if market['swap']}

In [76]:
# CALCUL DES INDICATEURS TECHNIQUES

async def indicators_calculation() :
    
    global data1
    global data2
    global data3 
    global data4 
    global data5
    global data6
    global data7
    global data8

    global top_10_values

    global fulldata
       
    fulldata_row = pd.DataFrame()

    index = [last_20.index[-1]] 

    data_row_step1 = pd.DataFrame({symbol + suffix:
                                   [last_20[symbol + " last"].iloc[-1]] if suffix == " last" else 
                                   [last_20[symbol + " ask"].iloc[-1]] if suffix == " ask" else 
                                   [last_20[symbol + " bid"].iloc[-1]] if suffix == " bid" else
                                   [(last_20[symbol + " last"].iloc[-1]) - (last_20[symbol + " last"].iloc[- wind_Var_Price])/
                                    last_20[symbol + " last"].iloc[- wind_Var_Price] * 100] if suffix == ' Var Last Price' and len(last_20) > wind_Var_Price else
                                     [ta.trend.ema_indicator(last_20[symbol + ' last'], window= wind_EMA1).iloc[-1]] if suffix == ' EMA_1' and len(last_20) > wind_EMA1 else
                                     [ta.trend.ema_indicator(last_20[symbol + ' last'], window= wind_EMA2).iloc[-1]] if suffix == ' EMA_2' and len(last_20) > wind_EMA2 else
                                     [ta.trend.ema_indicator(last_20[symbol + ' last'], window= wind_EMA3).iloc[-1]] if suffix == ' EMA_3' and len(last_20) > wind_EMA3 else
                                     [None] for symbol, ticker in tickers.items() for suffix in 
                                     [" last", " ask", " bid", ' Var Last Price', ' EMA_1', ' EMA_2', ' EMA_3']}, index=index)

                                
    data_row_step1 = data_row_step1.apply(pd.to_numeric)
    data1 = pd.concat([data1, data_row_step1])
    data1 = data1.tail(wind_Var_EMA1 + wind_Var_EMA2 + 30)
    # await asyncio.sleep(0.01)

   

    data_row_step2 = pd.DataFrame({symbol + suffix:
                                   (data1[symbol + " Var Last Price"].iloc[-1])**2 if suffix == ' Var Carre Last Price' and len(last_20) > wind_Var_Price else
                                   ((data1[symbol + ' EMA_1'].iloc[-1] - data1[symbol + ' EMA_1'].iloc[- wind_Var_EMA1]) /
                                    data1[symbol + ' EMA_1'].iloc[- wind_Var_EMA1]) * 100  if suffix == ' Var EMA_1' and len(last_20) > wind_EMA1 + wind_Var_EMA1  else
                                    ((data1[symbol + ' EMA_2'].iloc[-1] - data1[symbol + ' EMA_2'].iloc[- wind_Var_EMA2]) /
                                     data1[symbol + ' EMA_2'].iloc[- wind_Var_EMA2]) * 100  if suffix == ' Var EMA_2' and len(last_20) > wind_EMA2 + wind_Var_EMA2 else
                                     (data1[symbol + " last"].iloc[- 1] - data1[symbol + ' EMA_1'].iloc[-1]) ** 2 if suffix == ' Diff Carre Price_EMA_1' and len(last_20) > wind_EMA1 else
                                     (data1[symbol + ' EMA_1'].iloc[- 1] - data1[symbol + ' EMA_2'].iloc[-1]) ** 2 if suffix == ' Diff Carre EMA_1_2' and len(last_20) > wind_EMA2 else
                                     [None] for symbol, ticker in tickers.items() for suffix in 
                                     [' Var Carre Last Price', ' Var EMA_1', ' Var EMA_2', ' Diff Carre Price_EMA_1', ' Diff Carre EMA_1_2']}, index=index)
    
    data_row_step2 = data_row_step2.apply(pd.to_numeric)
    data2 = pd.concat([data2, data_row_step2])
    data2 = data2.tail(wind_Div_Price_EMA1 + wind_Div_EMA1_2 + 5)
    # await asyncio.sleep(0.01)
    
    data_row_step3 = pd.DataFrame({symbol+ suffix:
                                   ((data2[symbol + ' Diff Carre Price_EMA_1'].iloc[-1] - data2[symbol + ' Diff Carre Price_EMA_1'].iloc[- wind_Div_Price_EMA1]) /
                                    data2[symbol + ' Diff Carre Price_EMA_1'].iloc[- wind_Div_Price_EMA1]) * 100  if suffix == ' Divergence Price_EMA_1' and len(last_20) > wind_EMA1 + wind_Div_Price_EMA1 and data2[symbol + ' Diff Carre Price_EMA_1'].iloc[- wind_Div_Price_EMA1] != 0 else
                                    ((data2[symbol + ' Diff Carre EMA_1_2'].iloc[-1] - data2[symbol + ' Diff Carre EMA_1_2'].iloc[- wind_Div_EMA1_2]) /
                                    data2[symbol + ' Diff Carre EMA_1_2'].iloc[- wind_Div_EMA1_2]) * 100  if suffix == ' Divergence EMA_1_2' and len(last_20) > wind_EMA2 + wind_Div_EMA1_2 and data2[symbol + ' Diff Carre EMA_1_2'].iloc[- wind_Div_EMA1_2] != 0 else
                                    [None] for symbol, ticker in tickers.items() for suffix in 
                                    [' Divergence Price_EMA_1', ' Divergence EMA_1_2']}, index=index)
    
    data_row_step3 = data_row_step3.apply(pd.to_numeric)
    data3 = pd.concat([data3, data_row_step3])
    data3 = data3.tail(wind_Div_Moy_Price_EMA1 + wind_Div_Moy_EMA1_2 + 5)
    # await asyncio.sleep(0.01)

    data_row_step4 = pd.DataFrame({symbol + suffix:
                                   ta.trend.ema_indicator(data3[symbol + ' Divergence Price_EMA_1'], window= wind_Div_Moy_Price_EMA1).iloc[-1] if suffix == ' Divergence Moyen Price_EMA_1' and len(last_20) > wind_EMA1 + wind_Div_Price_EMA1 + wind_Div_Moy_Price_EMA1 else 
                                   ta.trend.ema_indicator(data3[symbol + ' Divergence EMA_1_2'], window= wind_Div_Moy_EMA1_2).iloc[-1] if suffix == ' Divergence Moyen EMA_1_2' and len(last_20) > wind_EMA2 + wind_Div_EMA1_2 + wind_Div_Moy_EMA1_2  else 
                                   [None] for symbol, ticker in tickers.items() for suffix in 
                                   [' Divergence Moyen Price_EMA_1', ' Divergence Moyen EMA_1_2']}, index=index)
    
    data_row_step4 = data_row_step4.apply(pd.to_numeric)
    data4 = pd.concat([data4, data_row_step4])
    data4 = data4.tail(wind_Div_Price_EMA1 + wind_Div_Moy_Price_EMA1 + wind_Div_EMA1_2 + wind_Div_Moy_EMA1_2 + 30)
    # await asyncio.sleep(0.01)

    data_row_step5 = pd.DataFrame({symbol + suffix:
                                   data4[symbol + ' Divergence Moyen Price_EMA_1'].iloc[-1] > 40000 and data4[symbol + ' Divergence Moyen EMA_1_2'].iloc[-1]  > 10000 if suffix == ' Condition'
                                   and len(last_20) > wind_EMA2 + wind_Div_EMA1_2 + wind_Div_Moy_EMA1_2 else
                                   [None] for symbol, ticker in tickers.items() for suffix in 
                                   [' Condition']}, index=index)
    
    # data_row_step5 = data_row_step5.apply(pd.to_numeric)
    data5 = pd.concat([data5, data_row_step5])
    # data5 = data5.apply(pd.to_numeric)
    data5 = data5.tail(5)
    # clear_output(wait=True)
    # display(data5.tail(2))
    # await asyncio.sleep(0.01)


    top_div = {}
    top_10_values = {}

    for symbol, ticker in tickers.items():
        if len(last_20) > wind_EMA1 + wind_Div_Price_EMA1:
            if data3[symbol + ' Divergence Price_EMA_1'].iloc[-1] > div_min_top:
                top_div[symbol] = (data3[symbol + ' Divergence Price_EMA_1'].iloc[-1], ticker['info']['symbol'])

    if top_div:
        top_10_values = dict(sorted(top_div.items(), key=lambda item: item[1][0], reverse=True)[:10])

    # display(f'top is : {top_10_values}')
  
    data_row_step6 = pd.DataFrame({symbol + suffix:
                                   1 if suffix == ' Top_10' and top_10_values is not None and symbol in top_10_values.keys() and len(last_20) > wind_EMA1 + wind_Div_Price_EMA1 else 
                                   [None] for symbol, ticker in tickers.items() for suffix in 
                                   [' Top_10']}, index=index)
    
    data_row_step6 = data_row_step6.apply(pd.to_numeric)
    data6 = pd.concat([data6, data_row_step6])
    data6 = data6.tail(occur_wind)
    # clear_output(wait=True)
    '''if len(last_20) > wind_EMA1 + wind_Div_Price_EMA1 :
        display(data6.tail(2))'''
    # await asyncio.sleep(0.01)

    data_row_step7 = pd.DataFrame({symbol + suffix: 
                                   data6[symbol + ' Top_10'].sum() if suffix == ' Occur' and len(fulldata) > wind_EMA1 + wind_Div_Price_EMA1 + wind_Div_Moy_Price_EMA1 + occur_wind + 1 else 
                                   [None] for symbol, ticker in tickers.items() for suffix in 
                                   [' Occur']}, index=index)
    data_row_step7 = data_row_step7.apply(pd.to_numeric)
    data7 = pd.concat([data7, data_row_step7])
    data7 = data7.tail(2)
    '''if len(fulldata) > wind_EMA1 + wind_Div_Price_EMA1 + wind_Div_Moy_Price_EMA1 + occur_wind :
        display(data7)'''
    # await asyncio.sleep(0.01)

    data_row_step8 = pd.DataFrame({symbol+ suffix:
                                   "open_long" if suffix == ' Tradable' and data7[symbol +' Occur'].iloc[-1] is not None and data7[symbol +' Occur'].iloc[-1] >
                                     min_occur and data5[symbol + ' Condition'].iloc[-1] and
                                   data1[symbol + ' Var Last Price'].iloc[-1] > 0 and data1[symbol + " last"].iloc[-1] >
                                     data1[symbol + ' EMA_1'].iloc[-1] > data1[symbol + ' EMA_2'].iloc[-1] > data1[symbol + ' EMA_3'].iloc[-1] and len(fulldata) >
                                       wind_EMA1 + wind_EMA2 + wind_Div_EMA1_2 + wind_Div_Moy_EMA1_2 + occur_wind else
                                   "open_short" if suffix == ' Tradable' and data7[symbol +' Occur'].iloc[-1] is not None and data7[symbol +' Occur'].iloc[-1] >
                                     min_occur and data5[symbol + ' Condition'].iloc[-1] and
                                   data1[symbol + ' Var Last Price'].iloc[-1] < 0 and data1[symbol + " last"].iloc[-1] <
                                     data1[symbol + ' EMA_1'].iloc[-1] < data1[symbol + ' EMA_2'].iloc[-1] < data1[symbol + ' EMA_3'].iloc[-1] and len(fulldata) >
                                       wind_EMA1 + wind_EMA2 + wind_Div_EMA1_2 + wind_Div_Moy_EMA1_2 + occur_wind else 
                                   [None] for symbol, ticker in tickers.items() for suffix in 
                                   [' Tradable']}, index=index) 
    data8 = pd.concat([data8, data_row_step8])
    data8 = data8.tail(2)
    '''if len(fulldata) > wind_EMA1 + wind_EMA2 + wind_Div_EMA1_2 + wind_Div_Moy_EMA1_2 + occur_wind :
        display(data8)'''
    # await asyncio.sleep(0.01)


    fulldata_row = pd.concat([data_row_step1, data_row_step2, data_row_step3, data_row_step4, data_row_step5, data_row_step6, data_row_step7, data_row_step8], axis=1)
    fulldata_row = fulldata_row.apply(pd.to_numeric, errors='ignore')
    fulldata_row =fulldata_row.sort_index(axis=1)
    fulldata = pd.concat([fulldata, fulldata_row])
    fulldata = fulldata.tail(wind_EMA1 + wind_EMA2 + wind_Div_EMA1_2 + wind_Div_Moy_EMA1_2 + occur_wind + wind_EMA2)
    # clear_output(wait=True)
    # display(data_row_step8)
    # await asyncio.sleep(0.01)

In [77]:
# MISE A JOUR DE LA BALANCE

async def refresh_balance() :

    global balance
    global free_balance
    global open_PNL
    global alertnbr
    global alert_df

    try :

        balance_data = await exchange.fetch_balance()

        await asyncio.sleep(0.005)

        balance = float(balance_data['info'][0]['available'])
        free_balance = float(balance_data['free']['USDT'])
        open_PNL = float(balance_data['info'][0]['unrealizedPL'])
    
    except Exception as e:
        
        alert = {'alert': str(e)}
        alert_data = pd.DataFrame(alert, index=[f'alert_nbr {alertnbr} balance'])
        alert_df = pd.concat([alert_df, alert_data])
        alertnbr += 1

    if balance < 150 :

        display("Balance is less than 150, stopping the program.")
        exit()

    # display(f'Balance is {balance}')
    # await asyncio.sleep(0.01)

In [78]:
# CHECK EXISTING Order & SL V2 test par list entière

async def config_strikerV2() :

    global df_open_trade_info
    global df_closed_trade_info
    global nbr_open_trade
    global trade_nb
    global nbr_max_trade
    global trail_stop
    global alertnbr
    global alert_df

    if not df_open_trade_info.empty:

        response_pos = None
        response_ord = None
        
        try : 

            response_pos = await exchange.fetch_positions(df_open_trade_info.index)

        except Exception as e: 
                            
            alert = {'alert': f'{e}'}
            alert_data = pd.DataFrame(alert, index=[f'GLOBAL WARNING POSITION {alertnbr} conf 22'])
            alert_df = pd.concat([alert_df, alert_data])
            alertnbr += 1
            
        if response_pos :

            for symbol in df_open_trade_info.index :

                if any("info" in pos and "symbol" in pos["info"] and pos["info"]["symbol"] == symbol for pos in response_pos) :

                    if df_open_trade_info.loc[symbol, 'status'] == "starting" or df_open_trade_info.loc[symbol, 'status'] == "set_new_sl" :

                        try :

                            params = {"stop" : True, "planType" : "track_plan"}
                            response_ord= await exchange.fetch_open_orders(symbol, params=params)
                            await asyncio.sleep(0.01)

                            if any('info' in resp and 'side' in resp['info'] and (
                                "close_long" in resp['info']['side'] or "close_short" in resp['info']['side']) for resp in response_ord) :
                                
                                df_open_trade_info.loc[symbol, 'status'] = "open"
                                
                            else :
                    
                                df_open_trade_info.loc[symbol, 'status'] = "SL WARNING" 

                        except Exception as e: 
                            
                            alert = {'alert': f'{e}'}
                            alert_data = pd.DataFrame(alert, index=[f'GLOBAL WARNING POSITION {alertnbr} {symbol}'])
                            alert_df = pd.concat([alert_df, alert_data])
                            alertnbr += 1
                            if df_open_trade_info.loc[symbol, 'status'] != "set_new_sl" :
                    
                                df_open_trade_info.loc[symbol, 'status'] = "set_new_sl" 

                            else :
                    
                                df_open_trade_info.loc[symbol, 'status'] = "SL WARNING"

                    if (df_open_trade_info.loc[symbol, 'status'] == "open" and df_open_trade_info.loc[symbol, 'sl_version'] == "high"
                        ) and ("info" in pos and "symbol" in pos["info"] and pos["info"]["symbol"] == symbol 
                               and pos["info"]['unrealizedPL']  < 0 for pos in response_pos) :
                        
                        df_open_trade_info.loc[symbol, 'sl_version'] = " set to low"

                    if (df_open_trade_info.loc[symbol, 'sl_version'] == "low" and "info" in pos and "symbol" in pos["info"] and
                         pos["info"]["symbol"] == symbol and pos["info"]['unrealizedPL'] > 0 for pos in response_pos) :
                        
                        df_open_trade_info.loc[symbol, 'sl_version'] = "set to high"
            
                '''else : 

                    if df_open_trade_info.loc[symbol, 'status'] == "open" :

                        try :

                            params = {"stop" : True, "planType" : "track_plan"}
                            await exchange.cancel_order(df_open_trade_info["sl_id"].loc[symbol], symbol, params=params)
                            await asyncio.sleep(0.01)
                            df_open_trade_info.loc[symbol, 'status'] = "first_closing"

                        except Exception as e: 
                            
                            alert = {'alert': f'{e}'}
                            alert_data = pd.DataFrame(alert, index=[f'WARNING closing SL {alertnbr} {symbol}'])
                            alert_df = pd.concat([alert_df, alert_data])
                            alertnbr += 1
                    
                            df_open_trade_info.loc[symbol, 'status'] = "close" 

                    elif df_open_trade_info.loc[symbol, 'status'] == "first_closing" :
                            
                        
                        response_ord = None
                        params = {"stop" : True, "planType" : "track_plan"}
                        response_ord= await exchange.fetch_open_orders(symbol, params=params)
                        await asyncio.sleep(0.01)

                        if not any('id' in resp  for resp in response_ord) :

                            df_open_trade_info.loc[symbol, 'status'] = "close" 

                        else : 

                            df_open_trade_info.loc[symbol, 'status'] = "SL WARNING CLOSING" '''

In [80]:
async def cleaner() :

    global df_open_trade_info
    global df_closed_trade_info
    global nbr_open_trade
    global trade_nb
    global nbr_max_trade
    global trail_stop

    if not df_open_trade_info.empty:

        for symbol in df_open_trade_info.index :

            if (df_open_trade_info.loc[symbol, 'status'] == "close" or df_open_trade_info.loc[symbol, 'status'] == "cancel" 
                or df_open_trade_info.loc[symbol, 'status'] == "bad-config" or df_open_trade_info.loc[symbol, 'status'] == "SL WARNING CLOSING"):
                
                df_closed_trade_info = pd.concat([df_closed_trade_info, df_open_trade_info.loc[[symbol]]])
                
                df_open_trade_info = df_open_trade_info.drop(symbol)

                nbr_open_trade -= 1
            
                await asyncio.sleep(0.005)

In [81]:
# MISE A JOUR SL 

async def sl_loader():

    global df_open_trade_info
    global df_closed_trade_info
    global nbr_open_trade
    global trade_nb
    global nbr_max_trade
    global trail_stop
    global alertnbr
    global alert_df

    if not df_open_trade_info.empty:

        for symbol in df_open_trade_info.index:

            list_change_sl = []

            if (df_open_trade_info.loc[symbol, 'status'] == "set_new_sl" or df_open_trade_info.loc[symbol, 'status'] == "SL WARNING" 
                or df_open_trade_info.loc[symbol, 'sl_version'] == "set to high" ):

                try : 
                
                    list_change_sl ={
                        'symbol': symbol,
                        'side': df_open_trade_info.loc[symbol, 'sl_side'],
                        'size': df_open_trade_info.loc[symbol, 'size'],
                        "marginCoin": "USDT",
                        'side': 'close_long',
                        "triggerType":"market_price",
                        "reduceOnly" : True,
                        "leverage": "1",
                        "planType" : "track_plan",
                        "rangeRate" : 0.2,
                        }
                
                    await asyncio.sleep(0.005)
                
                except Exception as e :
                            
                                alert = {'alert': str(e)}
                                alert_data = pd.DataFrame(alert, index=[f'Not alert_nbr {alertnbr} sl1'])
                                alert_df = pd.concat([alert_df, alert_data])
                                alertnbr += 1

                                if df_open_trade_info.loc[symbol, 'status'] != "SL WARNING" :
                    
                                    df_open_trade_info.loc[symbol, 'status'] = "SL WARNING"

                                else :
                    
                                    df_open_trade_info.loc[symbol, 'status'] = "SL WARNING 2"

            elif df_open_trade_info.loc[symbol, 'sl_version'] == "set to low" :

                try : 
                
                    list_change_sl ={
                        'symbol': symbol,
                        'side': df_open_trade_info.loc[symbol, 'sl_side'],
                        'size': df_open_trade_info.loc[symbol, 'size'],
                        "marginCoin": "USDT",
                        'side': 'close_long',
                        "triggerType":"market_price",
                        "reduceOnly" : True,
                        "leverage": "1",
                        "planType" : "track_plan",
                        "rangeRate" : 0.05,
                        }
                
                    await asyncio.sleep(0.005)
                
                except Exception as e :
                            
                                alert = {'alert': str(e)}
                                alert_data = pd.DataFrame(alert, index=[f'Not alert_nbr {alertnbr} sl2'])
                                alert_df = pd.concat([alert_df, alert_data])
                                alertnbr += 1

                                if df_open_trade_info.loc[symbol, 'status'] != "SL WARNING" :
                    
                                    df_open_trade_info.loc[symbol, 'status'] = "SL WARNING"

                                else :
                    
                                    df_open_trade_info.loc[symbol, 'status'] = "SL WARNING 2"

            if list_change_sl :

                try :
                        
                    new_sl = await exchange.privateMixPostPlanPlaceTrailStop(list_change_sl)

                    await asyncio.sleep(0.01)

                    if new_sl['msg'] == 'success' and 'orderId' in new_sl['data']:

                        df_open_trade_info.loc[symbol, "new_sl_id"] = new_sl['data']['orderId']

                        params = {"stop" : True, "planType" : "track_plan"}

                        respord = await exchange.fetch_open_orders(symbol, params= params)

                        if any("id" in resp and resp['id'] == df_open_trade_info.loc[symbol, 'sl_id'] and
                                'info' in resp and 'side' in resp['info'] and ("close_long" in resp['info']['side'] or "close_short" in resp['info']['side']) for resp in respord) :

                            try :
                                
                                params = {"stop" : True, "planType" : "track_plan"}

                                await exchange.cancel_order(df_open_trade_info["sl_id"].loc[symbol], symbol, params= params)

                                await asyncio.sleep(0.005)

                                df_open_trade_info.loc[symbol, 'sl_id'] = df_open_trade_info.loc[symbol, 'new_sl_id']

                                if df_open_trade_info.loc[symbol, 'sl_version'] == "set to high" :

                                    df_open_trade_info.loc[symbol, 'sl_version'] = "high"

                                if df_open_trade_info.loc[symbol, 'sl_version'] == "set to low" :

                                    df_open_trade_info.loc[symbol, 'sl_version'] = "low"

                            except Exception as e :
                            
                                alert = {'alert': str(e)}
                                alert_data = pd.DataFrame(alert, index=[f'Not alert_nbr {alertnbr} sl3'])
                                alert_df = pd.concat([alert_df, alert_data])
                                alertnbr += 1
                                    
                                df_open_trade_info.loc[symbol, 'sl_id'] = df_open_trade_info.loc[symbol, 'new_sl_id']
                                df_open_trade_info.loc[symbol, 'status'] = "SL WARNING"

                                display(f'trade SL {symbol} NOT IN EXCHANGE 1')

                except Exception as e :
                            
                    alert = {'alert': str(e)}
                    alert_data = pd.DataFrame(alert, index=[f'alert_nbr {alertnbr} sl4'])
                    alert_df = pd.concat([alert_df, alert_data])
                    alertnbr += 1

                    if df_open_trade_info.loc[symbol, 'status'] != "SL WARNING" :
                    
                        df_open_trade_info.loc[symbol, 'status'] = "SL WARNING"

                    else :
                    
                        df_open_trade_info.loc[symbol, 'status'] = "SL WARNING 2"

In [82]:
# OUVERTURE DES ORDRES

async def trade_opener() :

    global df_open_trade_info
    global df_closed_trade_info
    global nbr_open_trade
    global trade_nb
    global nbr_max_trade
    global trail_stop
    global alertnbr
    global alert_df

    if (top_10_values is not None and len(fulldata) > wind_EMA1 + wind_EMA2 + wind_Div_EMA1_2 + wind_Div_Moy_EMA1_2 + occur_wind 
        and not data8.empty and data8.iloc[-1].notnull().any() and free_balance > 50 and nbr_open_trade <= 14) :

        # display(f'for trade opener nbr_open_trade is {nbr_open_trade}')

        # for symbol, ticker in tickers.items() :
        for symbol in top_10_values.keys():
       
            trade = []

            # pprint.pprint(f'tentative pour {symbol}')
            
            if top_10_values[symbol][1] not in df_open_trade_info.index and data8[symbol + ' Tradable'].iloc[-1] == 'open_long' :

                # pprint.pprint(f'tentative data8 pour {symbol}')

                try :

                    trade = {
                        'symbol': top_10_values[symbol][1],
                        'side': 'open_long',
                        'size': exchange.amount_to_precision(symbol, round(balance * (0.9/nbr_max_trade), 2) / fulldata[symbol + ' last'].iloc[-1]),
                        'params' : {
                            "marginCoin": "USDT",
                            'side': 'open_long',
                            'force': "post_only",
                            "leverage": "1"}
                        }
                    
                    sl_trade = {
                        'symbol': top_10_values[symbol][1],
                        'side': 'close_long',
                        'size': exchange.amount_to_precision(symbol, round(balance * (0.9/nbr_max_trade), 2) / fulldata[symbol + ' last'].iloc[-1]),
                        "marginCoin": "USDT",
                        "triggerType":"market_price",
                        "reduceOnly" : True,
                        "leverage": "1",
                        "planType" : "track_plan",
                        "rangeRate" : 0.2,
                        }
                        
                
                except Exception as e:
                                     
                    alert = {'alert': str(e)}
                    alert_data = pd.DataFrame(alert, index=[f'alert_nbr {alertnbr} open1 {symbol}'])
                    alert_df = pd.concat([alert_df, alert_data])
                    alertnbr += 1
                    continue

                
                id = None
                sl_id = None
                response_create_sl = None
                response_open_ord = None
                create_ord = None

                try :
                    
                    response_create_sl = await exchange.privateMixPostPlanPlaceTrailStop(sl_trade)
                
                    await asyncio.sleep(0.01)

                except Exception as e:

                    alert = {'alert': str(e)}
                    alert_data = pd.DataFrame(alert, index=[f'alert_nbr {alertnbr} open7 {symbol}'])
                    alert_df = pd.concat([alert_df, alert_data])
                    alertnbr += 1
                    continue

                if response_create_sl ['msg'] == 'success' and 'orderId' in response_create_sl['data']:
                        
                    sl_id = response_create_sl['data']['orderId']
                        
                    await asyncio.sleep(0.01)

                    params = {"stop" : True, "planType" : "track_plan"}
                    response_open_ord = await exchange.fetch_open_orders(symbol, params= params)
                    await asyncio.sleep(0.01)
                    # response_open_ord.extend(await exchange.fetch_open_orders(symbol))
                    # await asyncio.sleep(0.01)

                    if len(response_open_ord) == 1 and any("id" in resp and resp["id"] == sl_id for resp in response_open_ord) : 

                        try :
                    
                            create_ord= await exchange.create_market_order(trade['symbol'], trade['side'], trade['size'], trade['params'])
                
                            await asyncio.sleep(0.01)

                        except Exception as e:
                                     
                            alert = {'alert': str(e)}
                            alert_data = pd.DataFrame(alert, index=[f'alert_nbr {alertnbr} open8 {symbol}'])
                            alert_df = pd.concat([alert_df, alert_data])
                            alertnbr += 1

                            try :

                                params = {"stop" : True, "planType" : "track_plan"}
                                await exchange.cancel_order(sl_id, symbol, params= params)
                                # await exchange.cancel_order(sl_id, symbol)

                            except Exception as e:

                                alert = {'alert': str(e)}
                                alert_data = pd.DataFrame(alert, index=[f'alert_nbr {alertnbr} open9 {symbol}'])
                                alert_df = pd.concat([alert_df, alert_data])
                                alertnbr += 1

                        if create_ord is not None :
                            
                            if "id" in create_ord : 
                        
                                id = create_ord['id']

                                await asyncio.sleep(0.005)

                                df_trade_row = pd.DataFrame({
                                    'symbol': [trade['symbol']],
                                    'size' : [float(trade['size'])],
                                    'side': [trade['side']],
                                    'opening_price': [fulldata[symbol + ' last'].iloc[-1]],
                                    'sl_side': [sl_trade['side']],
                                    'id': [id],
                                    "sl_id" : [sl_id],
                                    "new_sl_id" : "",
                                    'status' : "starting",
                                    'sl_version' : "high",
                                    "sym" : symbol
                                    })
                    
                                # pprint.pprint(df_trade_row)
                                df_trade_row.set_index('symbol', inplace=True)
                                # pprint.pprint(f'index is ok')
                                # df_trade_row['sl_triggerPrice'] = df_trade_row['sl_triggerPrice'].apply(pd.to_numeric)
                                df_open_trade_info = pd.concat([df_open_trade_info, df_trade_row])
                                # pprint.pprint(df_open_trade_info)
                                nbr_open_trade += 1
                                trade_nb += 1 

                                await refresh_balance()

                    elif len(response_open_ord) == 1 and response_open_ord['id'] != sl_id :


                            alert = {'alert': (f'!!! WARNING {symbol} OUT OF DF')}
                            alert_data = pd.DataFrame(alert, index=[f'ULTIME WARNING {alertnbr} open10 {symbol}'])
                            alert_df = pd.concat([alert_df, alert_data])
                            alertnbr += 1

            elif top_10_values[symbol][1] not in df_open_trade_info.index and data8[symbol + ' Tradable'].iloc[-1] == 'open_short':

                # pprint.pprint(f'tentative data8 pour {symbol}')
            
                try :

                    trade = {
                        'symbol': top_10_values[symbol][1],
                        'side': 'open_short',
                        'size': exchange.amount_to_precision(symbol, round(balance * (0.9/nbr_max_trade), 2) / fulldata[symbol + ' last'].iloc[-1]),
                        'params' : {
                            "marginCoin": "USDT",
                            'side': 'open_long',
                            'force': "post_only",
                            "leverage": "1"}
                        }
                    
                    sl_trade = {
                        'symbol': top_10_values[symbol][1],
                        'side': 'close_short',
                        'size': exchange.amount_to_precision(symbol, round(balance * (0.9/nbr_max_trade), 2) / fulldata[symbol + ' last'].iloc[-1]),
                        "marginCoin": "USDT",
                        "triggerType":"market_price",
                        "reduceOnly" : True,
                        "leverage": "1",
                        "planType" : "track_plan",
                        "rangeRate" : 0.2,
                        }
                        
                
                except Exception as e:
                                     
                    alert = {'alert': str(e)}
                    alert_data = pd.DataFrame(alert, index=[f'alert_nbr {alertnbr} open6 {symbol}'])
                    alert_df = pd.concat([alert_df, alert_data])
                    alertnbr += 1
                    continue

                
                id = None
                sl_id = None
                response_create_sl = None
                response_open_ord = None
                create_ord = None

                try :
                    
                    response_create_sl = await exchange.privateMixPostPlanPlaceTrailStop(sl_trade)
                
                    await asyncio.sleep(0.01)

                except Exception as e:

                    alert = {'alert': str(e)}
                    alert_data = pd.DataFrame(alert, index=[f'alert_nbr {alertnbr} open7 {symbol}'])
                    alert_df = pd.concat([alert_df, alert_data])
                    alertnbr += 1
                    continue

                if response_create_sl ['msg'] == 'success' and 'orderId' in response_create_sl['data']:
                        
                    sl_id = response_create_sl['data']['orderId']
                        
                    await asyncio.sleep(0.01)

                    params = {"stop" : True, "planType" : "track_plan"}
                    response_open_ord = await exchange.fetch_open_orders(symbol, params= params)
                    await asyncio.sleep(0.01)
                    # response_open_ord.extend(await exchange.fetch_open_orders(symbol))
                    # await asyncio.sleep(0.01)

                    if len(response_open_ord) == 1 and any("id" in resp and resp["id"] == sl_id for resp in response_open_ord) : 

                        try :
                    
                            create_ord= await exchange.create_market_order(trade['symbol'], trade['side'], trade['size'], trade['params'])
                
                            await asyncio.sleep(0.01)

                        except Exception as e:
                                     
                            alert = {'alert': str(e)}
                            alert_data = pd.DataFrame(alert, index=[f'alert_nbr {alertnbr} open8 {symbol}'])
                            alert_df = pd.concat([alert_df, alert_data])
                            alertnbr += 1

                            try :

                                params = {"stop" : True, "planType" : "track_plan"}
                                await exchange.cancel_order(sl_id, symbol, params= params)
                                # await exchange.cancel_order(sl_id, symbol)

                            except Exception as e:

                                alert = {'alert': str(e)}
                                alert_data = pd.DataFrame(alert, index=[f'alert_nbr {alertnbr} open9 {symbol}'])
                                alert_df = pd.concat([alert_df, alert_data])
                                alertnbr += 1

                        if create_ord is not None :
                            
                            if "id" in create_ord : 
                        
                                id = create_ord['id']

                                await asyncio.sleep(0.005)

                                df_trade_row = pd.DataFrame({
                                    'symbol': [trade['symbol']],
                                    'size' : [float(trade['size'])],
                                    'side': [trade['side']],
                                    'opening_price': [fulldata[symbol + ' last'].iloc[-1]],
                                    'sl_side': [sl_trade['side']],
                                    'id': [id],
                                    "sl_id" : [sl_id],
                                    "new_sl_id" : "",
                                    'status' : "starting",
                                    'sl_version' : "high",
                                    "sym" : symbol
                                    })
                    
                                # pprint.pprint(df_trade_row)
                                df_trade_row.set_index('symbol', inplace=True)
                                # pprint.pprint(f'index is ok')
                                # df_trade_row['sl_triggerPrice'] = df_trade_row['sl_triggerPrice'].apply(pd.to_numeric)
                                df_open_trade_info = pd.concat([df_open_trade_info, df_trade_row])
                                # pprint.pprint(df_open_trade_info)
                                nbr_open_trade += 1
                                trade_nb += 1 

                                await refresh_balance()

                    elif len(response_open_ord) == 1 and response_open_ord['id'] != sl_id :


                            alert = {'alert': (f'!!! WARNING {symbol} OUT OF DF')}
                            alert_data = pd.DataFrame(alert, index=[f'ULTIME WARNING {alertnbr} open10 {symbol}'])
                            alert_df = pd.concat([alert_df, alert_data])
                            alertnbr += 1

In [84]:
# PARAMETRAGE

nbr_max_trade = 15
time_to_fill = 45
min_occur = 3
div_min_top = 80000
trail_stop = 0.001

wind_Var_Price = 3
wind_EMA1 = 4
wind_Var_EMA1 = 3
wind_EMA2 = 9
wind_EMA3 = 17
wind_Var_EMA2 = 4
wind_Div_Price_EMA1 = 4
wind_Div_EMA1_2 = 6
wind_Div_Moy_Price_EMA1 = 2
wind_Div_Moy_EMA1_2 = 2

occur_wind = 20

trade_nb = 0
nbr_open_trade = 0
alertnbr = 1

# INITIALISATION DES DATAFRAMES

data1 = pd.DataFrame()
data2 = pd.DataFrame()
data3 = pd.DataFrame()
data4 = pd.DataFrame()
data5 = pd.DataFrame()
data6 = pd.DataFrame()
data7 = pd.DataFrame()
data8 = pd.DataFrame()
last_20 = pd.DataFrame()
alert_df = pd.DataFrame()
fulldata = pd.DataFrame()
df_open_trade_info = pd.DataFrame()
df_closed_trade_info = pd.DataFrame()

In [85]:
async def main():

    global wind_Var_Price 
    global wind_EMA1 
    global wind_Var_EMA1
    global wind_EMA2
    global wind_EMA3
    global wind_Var_EMA2
    global wind_Div_Price_EMA1
    global wind_Div_EMA1_2
    global wind_Div_Moy_Price_EMA1
    global wind_Div_Moy_EMA1_2

    global occur_wind
    global min_occur
    global div_min_top
    global trail_stop
    global nbr_max_trade
    global nbr_open_trade
    global trade_nb
    global alert_df
    global alertnbr
    global time_to_fill

    global last_20

    global data1
    global data2
    global data3 
    global data4 
    global data5
    global data6
    global data7
    global data8
    global fulldata
    global df_open_trade_info
    global df_closed_trade_info

    while True:

        try:

            await refresh_balance()
            display(last_20.index[-2:])
            display(df_closed_trade_info.tail(5))
            display(df_open_trade_info)
            display(f'Balance is {balance}')
            display(f'Open PNL is {open_PNL}')
            display(f'nbr open trade is {nbr_open_trade}')
            display(alert_df.tail(5))
            await config_strikerV2()
            await asyncio.sleep(0.005)
            await cleaner()
            await asyncio.sleep(0.005)
            await live_prices()
            await asyncio.sleep(0.005)
            await indicators_calculation()
            await asyncio.sleep(0.005)
            await sl_loader()
            await asyncio.sleep(0.005)
            await cleaner()
            await asyncio.sleep(0.005)
            await trade_opener()
            await asyncio.sleep(0.005)

        except KeyboardInterrupt:

            display(f'Interuption du programme ')
            await exchange.close()
            break

        clear_output(wait=True)

        await asyncio.sleep(0.001)

await main()

CancelledError: 

In [86]:
pprint.pprint(alert_df)

Empty DataFrame
Columns: []
Index: []


In [83]:
await exchange.fetch_positions(df_open_trade_info.index)

[{'info': {'marginCoin': 'USDT',
   'symbol': 'IDEXUSDT_UMCBL',
   'holdSide': 'long',
   'openDelegateCount': '0',
   'margin': '10.04011',
   'available': '179',
   'locked': '0',
   'total': '179',
   'leverage': '1',
   'achievedProfits': '0',
   'averageOpenPrice': '0.05609',
   'marginMode': 'crossed',
   'holdMode': 'double_hold',
   'unrealizedPL': '0.05728',
   'liquidationPrice': '-0.257284043689',
   'keepMarginRate': '0.02',
   'marketPrice': '0.05641',
   'marginRatio': '0.00250333333',
   'autoMargin': 'off',
   'cTime': '1702967235160',
   'uTime': '1702967235160'},
  'id': None,
  'symbol': 'IDEX/USDT:USDT',
  'notional': 10.09739,
  'marginMode': 'cross',
  'liquidationPrice': -0.257284043689,
  'entryPrice': 0.05609,
  'unrealizedPnl': 0.05728,
  'realizedPnl': None,
  'percentage': 0.57,
  'contracts': 179.0,
  'contractSize': 1,
  'markPrice': 0.05641,
  'lastPrice': None,
  'side': 'long',
  'hedged': True,
  'timestamp': 1702967235160,
  'datetime': '2023-12-19T06

In [47]:
params = {"stop" : True}
await exchange.fetch_open_orders('OCEANUSDT_UMCBL', params= params)

[{'info': {'orderId': '1121000233788575744',
   'clientOid': '1121000233784381442',
   'symbol': 'OCEANUSDT_UMCBL',
   'marginCoin': 'USDT',
   'size': '19',
   'executePrice': '0',
   'triggerPrice': '0',
   'status': 'not_trigger',
   'orderType': 'market',
   'planType': 'track_plan',
   'side': 'close_long',
   'triggerType': 'market_price',
   'presetTakeProfitPrice': '0',
   'presetTakeLossPrice': '0',
   'rangeRate': '0.2',
   'enterPointSource': 'API',
   'tradeSide': 'close_long',
   'holdMode': 'double_hold',
   'reduceOnly': True,
   'cTime': '1702993203589',
   'uTime': None},
  'id': '1121000233788575744',
  'clientOrderId': '1121000233784381442',
  'timestamp': 1702993203589,
  'datetime': '2023-12-19T13:40:03.589Z',
  'lastTradeTimestamp': None,
  'lastUpdateTimestamp': None,
  'symbol': 'OCEAN/USDT:USDT',
  'type': 'market',
  'timeInForce': 'IOC',
  'postOnly': None,
  'side': 'sell',
  'price': None,
  'stopPrice': 0.0,
  'triggerPrice': 0.0,
  'average': None,
  'cos